In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk

In [3]:
train=pd.read_csv('labeledTrainData.tsv',header=0, delimiter="\t", quoting=3)

In [4]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [6]:
train.shape

(25000, 3)

In [7]:
train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [40]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

----------------------------------------------------------------

In [25]:
review=BeautifulSoup("<br>hi I am playing play with you loves loved ;) :-) :)").get_text()

In [26]:
emoctions=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
emoctions

[u';)', u':-)', u':)']

In [27]:
letters_only = re.sub("[^a-zA-Z]", " ", review)  + ' '.join(emoctions).replace('-', '')

In [28]:
letters_only

u'hi I am playing play with you loves loved          ;) :) :)'

In [29]:
words = letters_only.lower().split()
words

[u'hi',
 u'i',
 u'am',
 u'playing',
 u'play',
 u'with',
 u'you',
 u'loves',
 u'loved',
 u';)',
 u':)',
 u':)']

In [30]:
stops = stopwords.words("english")

In [31]:
meaningful_words = [w for w in words if not w in stops] 
meaningful_words

[u'hi', u'playing', u'play', u'loves', u'loved', u';)', u':)', u':)']

In [32]:
porter = PorterStemmer()
meaningful_words=[porter.stem(word) for word in meaningful_words]
meaningful_words

[u'hi', u'play', u'play', u'love', u'love', u';)', u':)', u':)']

In [33]:
" ".join( meaningful_words )

u'hi play play love love ;) :) :)'

-----------------------------------------------------------------

In [43]:
def review_to_words(raw_review):
    
    # 1. remove htmls
    review_text=BeautifulSoup(raw_review).get_text()
    
    emoctions=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review_text)
    
    # 2. remove non letters
    letters_only=re.sub("[^a-zA-Z]"," ",review_text) + ' '.join(emoctions).replace('-','')
    
     # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    
    # 4. Remove stop words
    
    stops = stopwords.words("english")
    meaningful_words = [w for w in words if not w in stops]
    
    #porter = PorterStemmer()
    #[porter.stem(word) for word in meaningful_words]
    
    snowball_stemmer = SnowballStemmer('english')
    meaningful_words=[snowball_stemmer.stem(word) for word in meaningful_words]
    
    # 5. Join the words back into one string separated by space and return the result.
    return( " ".join( meaningful_words ))
    

In [44]:
clean_review = review_to_words( train["review"][0] )
print clean_review

stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obvious messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl stay 

In [45]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

print "Cleaning and parsing the training set movie reviews...\n"
clean_train_reviews = []

for i in xrange( 0, num_reviews ):
    
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews )   
        
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [46]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [47]:
 print train_data_features.shape

(25000L, 5000L)


In [48]:
vocab = vectorizer.get_feature_names()
print vocab

[u'abandon', u'abc', u'abil', u'abl', u'abomin', u'aborigin', u'abort', u'abound', u'abraham', u'abrupt', u'absenc', u'absent', u'absolut', u'absorb', u'absurd', u'abund', u'abus', u'abysm', u'academi', u'accent', u'accept', u'access', u'accid', u'accident', u'acclaim', u'accompani', u'accomplish', u'accord', u'account', u'accur', u'accuraci', u'accus', u'ace', u'achiev', u'acid', u'acknowledg', u'acquaint', u'acquir', u'across', u'act', u'action', u'activ', u'actor', u'actress', u'actual', u'ad', u'adam', u'adapt', u'add', u'addict', u'addit', u'address', u'adequ', u'adjust', u'admir', u'admit', u'adolesc', u'adopt', u'ador', u'adult', u'advanc', u'advantag', u'adventur', u'advertis', u'advic', u'advis', u'aesthet', u'affair', u'affect', u'affleck', u'afford', u'aforement', u'afraid', u'africa', u'african', u'afternoon', u'afterward', u'age', u'agenc', u'agenda', u'agent', u'aggress', u'ago', u'agre', u'ah', u'ahead', u'aid', u'aim', u'air', u'airplan', u'airport', u'aka', u'akin', u'

In [49]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

288 abandon
125 abc
562 abil
1259 abl
83 abomin
69 aborigin
92 abort
63 abound
93 abraham
136 abrupt
118 absenc
83 absent
1850 absolut
154 absorb
427 absurd
73 abund
398 abus
110 abysm
298 academi
704 accent
781 accept
165 access
344 accid
246 accident
118 acclaim
197 accompani
271 accomplish
329 accord
297 account
349 accur
82 accuraci
204 accus
75 ace
578 achiev
102 acid
109 acknowledg
73 acquaint
97 acquir
971 across
8794 act
3694 action
268 activ
6876 actor
1588 actress
5065 actual
793 ad
414 adam
835 adapt
1147 add
261 addict
499 addit
183 address
148 adequ
59 adjust
440 admir
875 admit
112 adolesc
162 adopt
230 ador
887 adult
275 advanc
172 advantag
773 adventur
228 advertis
262 advic
197 advis
78 aesthet
419 affair
430 affect
66 affleck
139 afford
126 aforement
343 afraid
212 africa
284 african
197 afternoon
183 afterward
1726 age
79 agenc
86 agenda
455 agent
103 aggress
1033 ago
779 agre
119 ah
396 ahead
289 aid
325 aim
842 air
106 airplan
96 airport
195 aka
67 akin
125 akshay


In [50]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

In [51]:
# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

(25000, 2)


In [52]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"

for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [53]:
result = forest.predict(test_data_features)

In [54]:
result

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [55]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_rf_model.csv", index=False, quoting=3 )

-----------------------------------

In [56]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [57]:
nb=nb.fit(train_data_features, train["sentiment"])

In [58]:
result_nb = nb.predict(test_data_features)

In [59]:
result_nb

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [60]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result_nb} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_nb_model.csv", index=False, quoting=3 )

------------------------------

In [61]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
stop = stopwords.words('english')

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [62]:
gs_lr_tfidf.fit(clean_train_reviews, train["sentiment"] )

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0], 'vect__stop_words': [[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her',...orm': [None], 'vect__use_idf': [False], 'clf__C': [1.0, 10.0, 100.0], 'clf__penalty': ['l1', 'l2']}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1

In [63]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'vect__ngram_range': (1, 1), 'clf__penalty': 'l2', 'clf__C': 10.0, 'vect__stop_words': [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'ho

In [64]:
print gs_lr_tfidf.best_score_

0.88664


In [65]:
clf = gs_lr_tfidf.best_estimator_

In [66]:
clf.fit(clean_train_reviews, train["sentiment"])

Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [67]:
result_lr=clf.predict(clean_test_reviews)

In [69]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result_lr} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model_two.csv", index=False, quoting=3 )

--------------------------------------

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)

In [77]:
train_data_features_tfidf=tfidf.fit_transform(clean_train_reviews).toarray()

In [79]:
train_data_features_tfidf.shape

(25000L, 5000L)

In [89]:
test_data_features_tfidf = tfidf.transform(clean_test_reviews)
test_data_features_tfidf = test_data_features_tfidf.toarray()

In [90]:
forest_tfidf = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest_tfidf = forest.fit( train_data_features_tfidf, train["sentiment"] )

In [91]:
result_tfidf = forest_tfidf.predict(test_data_features_tfidf)

In [92]:
result_tfidf

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [93]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result_tfidf} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model_tfidf.csv", index=False, quoting=3 )